In [8]:
using Pkg; Pkg.activate(".")
# filter(x -> occursin("SLURM", x), keys(ENV))
#ENV["SLURM_NNODES"]
#parse(Int, ENV["SLURM_JOB_CPUS_PER_NODE"])

  Activating project at `/central/home/jschmitt/ClimaAtmos.jl/calibration/experiments/gcm_driven_scm`


In [9]:
# use in single threaded mode
import ClimaAtmos as CA
import ClimaAnalysis: SimDir, get, slice, average_xy
import CairoMakie
import JLD2
import LinearAlgebra: I
import EnsembleKalmanProcesses as EKP
import Statistics: var, mean
using Test
using Revise

include("observation_map.jl")

using Revise
import ClimaCalibrate as CAL
import ClimaAtmos as CA
experiment_dir = joinpath(pkgdir(CA), "calibration", "experiments", "gcm_driven_scm")
model_interface =
    joinpath(pkgdir(CA), "calibration", "model_interface.jl")
output_dir = joinpath("output", "gcm_driven_scm")
# include model interface
includet(model_interface)
obs_path = joinpath(experiment_dir, "observations.jld2")


"/central/home/jschmitt/ClimaAtmos.jl/calibration/experiments/gcm_driven_scm/observations.jld2"

In [10]:
observations = JLD2.load_object(obs_path)
noise = 0.1 * I
n_iterations = 3
ensemble_size = 9
prior = CAL.get_prior(joinpath(experiment_dir, "prior.toml"))
experiment_config = CAL.ExperimentConfig(;
    n_iterations,
    ensemble_size,
    observations,
    noise,
    output_dir,
    prior,
)

ExperimentConfig(3, 9, [285.0], LinearAlgebra.UniformScaling{Float64}(0.1), ParameterDistribution with 5 entries: 
'mixing_length_eddy_viscosity_coefficient' with EnsembleKalmanProcesses.ParameterDistributions.Constraint{EnsembleKalmanProcesses.ParameterDistributions.BoundedBelow}[Bounds: (0.0, ∞)] over distribution EnsembleKalmanProcesses.ParameterDistributions.Parameterized(Distributions.Normal{Float64}(μ=-1.9686573911265683, σ=0.07133771448169109)) 
'detr_massflux_vertdiv_coeff' with EnsembleKalmanProcesses.ParameterDistributions.Constraint{EnsembleKalmanProcesses.ParameterDistributions.BoundedBelow}[Bounds: (0.0, ∞)] over distribution EnsembleKalmanProcesses.ParameterDistributions.Parameterized(Distributions.Normal{Float64}(μ=0.13599544622138604, σ=0.0867928125339026)) 
'entr_coeff' with EnsembleKalmanProcesses.ParameterDistributions.Constraint{EnsembleKalmanProcesses.ParameterDistributions.BoundedBelow}[Bounds: (0.0, ∞)] over distribution EnsembleKalmanProcesses.ParameterDistribut

In [11]:
obs_path = joinpath(experiment_dir, "observations.jld2")
observations = JLD2.load_object(obs_path)
observations

1-element Vector{Float64}:
 285.0

In [ ]:
@everywhere begin
    using JLD2
    experiment_dir = dirname(Base.active_project())
    obs_path = joinpath(experiment_dir, "observations.jld2")
    if !isfile(obs_path)
        error("Observation file not found at: $obs_path")
    else
        println("Worker ", myid(), " can access the observations file.")
    end
end

In [ ]:
reshape(G_ens[1,:], 1, 10)
EKP.update_ensemble!(eki, reshape(G_ens[1,:], 1, 10))
CAL.save_eki_state(eki, output_dir, 1, prior)